In [22]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import mysql.connector as connector

In [23]:
#!pip install mysql-connector-python

## Importing data using yfinance

In [24]:
stock = yf.Ticker("NVDA") 

In [25]:
stock_historical = stock.history(period="max", interval="1d")
stock_historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1999-01-22 00:00:00-05:00,0.040122,0.044779,0.035585,0.037615,2714688000,0.0,0.0
1999-01-25 00:00:00-05:00,0.040600,0.042033,0.037615,0.041556,510480000,0.0,0.0
1999-01-26 00:00:00-05:00,0.042033,0.042869,0.037734,0.038331,343200000,0.0,0.0
1999-01-27 00:00:00-05:00,0.038451,0.039406,0.036301,0.038212,244368000,0.0,0.0
1999-01-28 00:00:00-05:00,0.038212,0.038451,0.037854,0.038092,227520000,0.0,0.0
...,...,...,...,...,...,...,...
2025-02-24 00:00:00-05:00,136.559998,138.589996,130.080002,130.279999,251381100,0.0,0.0
2025-02-25 00:00:00-05:00,129.979996,130.199997,124.440002,126.629997,271428700,0.0,0.0
2025-02-26 00:00:00-05:00,129.990005,133.729996,128.490005,131.279999,322553800,0.0,0.0


In [26]:
df = pd.DataFrame(stock_historical)

## SQL Connection

In [27]:
try:
    conn = connector.connect(user='root', password='123456', host='localhost', port='3306', database='my_db')
except connector.Error as er:
    print('Error code:', er.errorno)
    print('Error msg:', er.msg)

In [28]:
mycursor = conn.cursor()

mycursor.execute("SHOW TABLES")

tables = [table[0] for table in mycursor.fetchall()]

if 'stock_price' not in tables:
    mycursor.execute("CREATE TABLE stock_price (Date DATE UNIQUE, Open FLOAT(24), High FLOAT(24), Low FLOAT(24), Close FLOAT(24), Volume FLOAT(24), Dividends FLOAT(24), Stock_Splits FLOAT(24))")
    print("Table 'stock_price' created successfully.")
else:
    print("Table 'stock_price' already exists.")


Table 'stock_price' already exists.


In [30]:
df.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,6567.000000,6567.000000,6567.000000,6567.000000,6.567000e+03,6567.000000,6567.000000
mean,8.941228,9.104114,8.757632,8.937180,5.986602e+08,0.000028,0.003274
std,24.358276,24.790504,23.830547,24.322010,4.306135e+08,0.000357,0.140806
min,0.032003,0.032600,0.030569,0.031286,1.968000e+07,0.000000,0.000000
25%,0.257701,0.264832,0.250794,0.258045,3.379760e+08,0.000000,0.000000
50%,0.437238,0.444022,0.430877,0.437603,5.000700e+08,0.000000,0.000000
75%,4.628212,4.712202,4.570358,4.632378,7.305060e+08,0.000000,0.000000
max,153.029999,153.130005,147.820007,149.429993,9.230856e+09,0.010000,10.000000


In [31]:
#df.set_index('Date', drop=True, inplace=True)


## Inserting data into sql table stock_price

In [32]:
#sql ="INSERT INTO"

sql = "INSERT IGNORE INTO stock_price (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

# Convert DataFrame to list of tuples without the index
tuples_list = list(df.itertuples(index=True, name=None))

#need to use executemany to insert multiple rows
mycursor.executemany(sql, tuples_list)

In [33]:
#commit from the connection
conn.commit()

mycursor.close()
conn.close()